# **UCF Computer Vision Lectures 2014**

These are my notes taken while watching the 2014 computer vision lectures by Mubarak Shah on Youtube.

## **Lecture 1: Introduction to Computer Vision**



The Lambertian model involves the assumption that light reflected on a rough surface is only dependent on the surface normal and the light direction. This will be violated by most objects as they will have specularities which will be view dependent.

Because we project the 3D world to the 2D image, we lose depth information. There are many techniques that attempt to recover this information that go by the name "Shape from X" like shape from stereo, shape from motion, shape from shading, shape from texture, shape from contours etc. The human vision system is able to perceive depth because there are two eyes (stereo), motion, and other cues. 

## **Lecture 2: Filtering**


A image histogram gives the number of pixels at each grey level. 

The Gaussian filter is an averaging/smoothing filter given by $g(x,y) = e^{-\frac{x^2 + y^2}{2\sigma^2}}$. It is a separable filter.

## **Lecture 3: Edge Detection**

Intensity or color changes non-continuously at an edge. Edges are caused due to a variety of factors including *surface normal discontinuity*, *depth discontinuity*, *surface color discontinuity*, and *illumination discontinuity*. The derivatives in a noisy image can vary widely since a pixel will look very different from its neighbors, so we smooth the image with a Gaussian first before detecting for edges. 


In the Prewitt and Sobel edge detectors we will find the derivatives in $x$ and $y$ directions and threshold the magnitude of the computed gradient. In the Marr-Hildreth Edge detector we smooth the image with a Gaussian filter and then calculate the laplacian. This means that instead looking for a suitable threshold of the magnitude of the gradient, we would only need to look for zero crossings of the laplacian. But, in practice, we will actually need to set a threshold. Given a zero-crossing from $a$ to $-b$, we will consider the value $\lvert a + b \rvert$ and threshold based on this value. The Laplacian-of-Gaussian (LoG) filter is separable with 4 1-D filters.

The Canny Edge Detector smooths the image, computes the magnitude and orientation of the gradient, applies *non-maximal suppression*, and then applies a *hysteresis threshold*. This was the first filter that considered the *orientation* of the gradient in addition to the magnitude. In the non-maximal suppression step, a pixel that doesn't have a greater gradient than its neighbors in the direction of the gradient is suppressed. In the hysteresis threshold step, we select those pixels which are above a high threshold as an edge pixel, and those that are between the high and low threshold are selected if they are connected to a pixel above the high threshold.



## **Lecture 4: Interest Points Detection**

An interest point is expressive in texture at which the direction of the boundary changes abruptly. These points lie at the intersection of two or more edge segments. Interest points allow us to correspond between two views and potentially estimate the fundamental matrix.

 Some possible approaches to corner detection are based on:

* Brightness of images (image derivatives), or
* Boundary extraction. Here edges are first computed and then the curvature of the edges is analyzed.

Interest Point detections are useful in:

* Automating object tracking.
* Point matching for computing disparity
* Stereo calibration
* Motion based segmentation
* Recognition
* 3D object recognition
* Robot navigation


The basic keypoint matching algorithm follows the following steps:

1. Find a set of distinctive key-points.
2. Define a region around each point.
3. Extract and normalize the region content.
4. Compute a local descriptor from the normalized region.

While detecting interest points, there is a tradeoff between being more repeatable (precise localization and robust detection) and obtaining more points (robust to occlusion).

Ultimately, we would like to identify patches near interest points and matching these in different views.


### **The Harris Corner Detector**

We would like to find a window around the corner point such that shifting the image in any direction leads to a change in intensity. 


The change of intensity for the shift $(u,v)$ is $$E(u,v)  = \sum_{x,y} \left(I(x+u, y+v) - I(x,y)\right)^2$$ Using the taylor series, we can approximate $I(x + u, y+v) \approx I(x, y) + I_x(x,y)u + I_y(x,y)v$. So we get $$E(u,v) \approx \sum_{x,y} (u I_x + vI_y)^2 = \sum_{x,y} \begin{bmatrix}u & v \end{bmatrix}\begin{bmatrix} I_x \\ I_y\end{bmatrix}\begin{bmatrix}I_x & I_y \end{bmatrix}\begin{bmatrix}u & v \end{bmatrix} = \begin{bmatrix} u & v \end{bmatrix} M \begin{bmatrix}u \\ v \end{bmatrix} $$ where $M = \sum_{x,y} \begin{bmatrix} I_x \\ I_y\end{bmatrix}\begin{bmatrix} I_x & I_y \end{bmatrix}$. 

$E(u,v)$ is a quadratic form, and so describes an ellipse. Since $M$ is symmetric, there are two real eigenvalues $\lambda_1 \geq \lambda_2 \geq 0$ representing the maximum and minimum possible growth of $E(u,v)$ for unit vectors $[u,v]^T$, respectively. We would like there to be a large change in every direction in a corner, so we would like both the eigenvalues to be large (bigger than some threshold value). If there is one eigenvalue that is much bigger than the other, then it is likely an edge.

It is computationally intensive to compare the eigenvalues directly, so we will instead compare the value of $R = \det(M) - \text{trace}(M)^2$. We will have that $R > 0$ for $\lambda_1$ and $\lambda_2$ large positive values. 

In the harris correlation,  we usually have a window function $w(x,y)$ as well, but this doesn't change any of the calculations. We will only have $M = \sum_{x,y} w(x,y)\begin{bmatrix} I_x \\ I_y\end{bmatrix}\begin{bmatrix} I_x & I_y \end{bmatrix}$. A common choice is a Gaussian window function.

If $I \to aI +b$, then we may have new corners detected if we have the same threshold value. We will perform non-maximal suppression for this reason.

The Harris corner detector is not scale invariant. We would like to make it scale invariant, so we can consider regions of different sizes around the point. Regions of corresponding sizes will look the same in both images. Then we would like to design a function on a region that is scale invariant. One option is to compute the maximal $R$ value for different scales and pick the maximum of these values. 

The Laplacian of Gaussian LoG filter is also called a *blob detector*. We can approximate the second derivative in 1D by the fomula $\frac{f(x + h)  - 2f(x) + f(x-h)}{h^2}\approx  f^{''}(x)$. In 2D, this corresponds to the mask $\begin{bmatrix} 0 & 1 & 0 \\ 1 & -4 & 1\\ 0 & 1 & 0 \end{bmatrix}$

## **Lecture 5: Image Pyramids**

Image Pyramids are useful for representing images. They allow us to detect objects at different scales, but are also useful in image compression and image stitching. Pyramids are built by using multiple copies of an image at a certain resolution. Each level in the pyramid is $\frac{1}{4}$ th the size of the previous level. The lowest level is the highest resolution, while the highest level is the lowest resolution. Two common types of pyramids include *Gaussian pyramids* and *Laplacian pyramids*. Gaussian pyramids are pyramid of images, but laplacian pyramids are pyramids of edges. 


The reduction step in a Gaussian pyramid at level $\ell$ is given by $$g_{\ell}(i,j) = \sum_{m = -2}^2 \sum_{n = -2}^2 w(m,n) g_{\ell-1}(2i+m, 2j+n)$$ while the expand step is given $$g_{\ell, n}(i,j) = \sum_{p = -2}^2 \sum_{q=-2}^2 w(p,q) g_{\ell, n-1}\left(\frac{i-p}{2}, \frac{j-q}{2}\right)$$

If we want weights that are symmetric and separable, we will have to choose a Gaussian filter.

The Laplacian pyramid is obtained from the Gaussian pyramid. We have $L_{\ell} = g_{\ell} - \verb|expand|(g_{\ell+1})$. In the laplacian pyramid, most edges are highlighted. To decode the laplacian pyramid, we can compute the Gaussian pyramid from the Laplacian pyramid. We will have $g_\ell = \verb|expand|(g_{\ell + 1}) + L_{\ell}$. We can stitch images together by combining laplacian pyramids of the two images together.

## **Lecture 6: SIFT Detector and Descriptor**

SIFT stands for *Scale Invariant Feature Transform* and it is used to extract distinctive invariant features. These features are invariant under scale and rotation, as well as robust to changes in 3D viewpoint, noise, affine distortion, and change in illumination.

We would like to find points in image that can be precisely and reliably found in other images. This will help compute the fundamental matrix, for example. To ensure this is useful, we would like a *repeatable* detector (so that the same point is found in multiple views) that is also a *descriptor* (so that we can reliably find the correct point in another image). 

Feature points are also used for:

* Image alignment (homography or fundamental matrix)
* 3D reconstruction
* Motion tracking
* Object recognition
* Indexing and database retrieval
* Robot navigation, etc.

The advantages of SIFT are:

* **Locality**: the features are local, so SIFT is robust to occlusion and clutter.
* **Distinctiveness**: individual features can be matched to a large database of objects
* **Quantity**: many features can be generated even for small objects.
* **Efficiency**: there is close to real-time performance.

The main steps for extracting the ley points are (SIF points):

1. *Scale space peak selection*. The peaks will be potential features. 
2. *Key point localization*.
3. *Orientation assignment*. The orientation will be like a descriptor.
4. *Key point descriptor*. The key point is described as a high dimensional vector ($\mathbb{R}^{128})$.

LoG can be efficiently approximated by a difference of Gaussians at different scales. $G(x,y, k\sigma) - G(x,y,\sigma) \approx \verb|LoG|(x,y; \sigma)$.

We would like to find robust maxima and minima in space and scale to localize key points.

We will compute difference of gaussians in space and scales. One way to use the scale space was to form a DoG pyramid to find maximal values, and then eliminating any edges detected. For each pixel, we have  $8$ neighbors in the same scale, 9 neighbors above and 9 below (in the DoG pyramid). If a pixel is a maximum among these neighboring pixels, we pick it.

This SIFT detector is invariant under scales. It uses difference of Gaussians in scale and space. 


We would like to build a *descriptor*, not only a detector, so we are able to describe the neighborhood of our points. Nowadays, Harris-Laplace or some other method is used for keypoint localization, but the SIFT descriptor is still in use.

A basic solution might be to just pick a neighborhood of each feature at the appropriate scale, but this has issues. Correlation is not invariant under rotation or photometric changes.

The image content is represented by a "constellation" of local features that are invariant under rigid motion, scale, and other imaging parameters.

We will need to assign an orientation for the key point and use that to describe it. Then we want to build a descriptor for the keypoint by using local image gradients at the selected scale and rotation to describe each keypoint region.

We find the dominant direction of the image gradient, and choose that as the base orientation. Then we create a histogram of local histogram directions at selected scale and pick the canonical orientation to be the peak of the histogram in  this neighborhood.

Before building the descriptor, we normalize by rotating the window to standard orientation, and scaling the window size based on the scale at which the point is found. There are a lot of details involved in getting SIFT to work, so we normally just use libraries for descriptors.

To build our descriptor, we build a histogram of gradients (HoG) weighted by a gaussian. Then we form a $4 \times 4$ array, each of $8$ bins (for the 8 possible directions). In fact, there is tri-linear interpolation so that each pixel contributes to multiple histograms. This means that a small change in the window doesn't change the descriptor greatly.

In order to reduce the effects of illumination changes, we clip the gradient magnitudes to avoid excessive influence of high gradients (for example, after normalization, clamp gradients bigger than $0.2$). We also normalize the vector to magnitude $1.0$.

To match features, we use a k-D tree and a best bin first algorithm.